In [145]:
from urllib.request import urlopen
from selenium import webdriver as wd 
from bs4 import BeautifulSoup
from tqdm import tqdm

import itertools
import pandas as pd
import numpy as np
import requests
import time
import copy
import re

## 현재 존재하는 아이템 리스트 가져오기

In [39]:
def make_itemlist():
    url = 'https://lolchess.gg/items'
    hdr = {'Accept-Language' : 'Ko_KR', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
    req = requests.get(url, headers = hdr)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')

    itemdict = {}
    for i in soup.find_all('div',{'class':'col-sm-6'}):
        arr = []
        for index, item in enumerate(i.select('img')):
            if index == 2:
                itemdict[item.get('alt')] = arr[:]
            else:
                arr.append(item.get('alt'))
        arr.clear()
    return itemdict

## 아이템 리스트 불러오기

In [40]:
item_list = make_itemlist()
a = set()
for key, value in item_list.items():
    for i in value:
        a.add(i)
a = list(a)
a.sort()
small_itemlist=a

## 합친 아이템을 분해해서 구성요소를 볼 수 있는 함수

In [6]:
def decomposition_item(item_list,arr):
    small_item = []
    for full in arr:
        if full in item_list:
            small_item.append(item_list[full][0])
            small_item.append(item_list[full][1])
        else:
            small_item.append(full)
        small_item.sort()

    return small_item

## Beautiful Soup을 이용하여서 상위 티어의 아이템/ 시너지/ 승률 가져오기

In [7]:
# url을 이용하여서 html을 받아오는 함수
def rcv_sp(hdr, url, check):
    req = requests.get(url, headers = hdr)
    sp = BeautifulSoup(req.text, 'html.parser')
    
    if check == 1:
        return sp
    else : 
        return sp.find('div',{'class' : 'profile__match-history-v2__filters'}).select('a')[1]['href']

#해당 게임의 아이템을 받아오는 함수     
def rcv_items(itemsp):
    itemlist = []
    for i in itemsp:
        tp = i.select('.item')
        if len(tp) == 0:
            continue
        for items in tp:
            itemlist.append(items['title'])
    return itemlist

#해당 게임의 조합을 받아오는 함수
def rcv_traits(traitsp):
    traitslist = []
    for i in traitsp:
        traitslist.append(i.find('img')['alt'])
        
    return traitslist
    
def main_func():
    url = 'https://lolchess.gg/leaderboards'
    hdr = {'Accept-Language' : 'Ko_KR', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
    req = requests.get(url, headers = hdr)
    soup = (BeautifulSoup(req.text,'html.parser'))
    #데이터를 저장할 배열
    save_data = []
    for i in tqdm(soup.find('tbody').select('a')):
        #처음에 i 에서 유저의 정보를 받아서 이 유저의 랭크의 soup를 받아온다
        sp = rcv_sp(hdr,rcv_sp(hdr,i['href'],0),1)
        #하나씩 저장
        temp_data = []
        for j in sp.select('.profile__match-history-v2__item'):
            rank = j.find('div',{'class':'placement'}).text.strip()[1]
            traits = rcv_traits(j.select('.tft-hexagon-image'))
            items = rcv_items(j.select('.items'))
            temp_data.append(rank)
            temp_data.append(traits)
            temp_data.append(items)
            save_data.append(temp_data[:])
            temp_data.clear()
    return save_data    
    

In [8]:
data = main_func()

100%|██████████| 100/100 [02:31<00:00,  1.51s/it]


## 아이템의 갯수를 비교해서 가장 근접한 조합을 넣는다. 

In [79]:
def match_item(final_set, temp):
    tp = []
    max_score = 0
    now = -1
    for i, v in enumerate(final_set):
        score = 0
        total = 0
        for value in temp:
            s = int(value[0])
            total += s
            item = value[1:]
            for it in v[1]:
#                 print(f'it : {it[1:]}, item : {item}')
                if it[1:] == item:
                    score += min(s,int(it[0]))
                    break
        if max_score < score:
            max_score = score
            now = i
    
    if now != -1:
        tp.append(final_set[14][now])
        tp.append(round(max_score/total,2))
    else:
        tp.append(0)
        tp.append(0)
    
    return tp

In [154]:
cp = copy.deepcopy(data)
for i in cp:
    full_item = i.pop()
    small_item = decomposition_item(item_list,full_item)
    traits = i.pop()
    
    sm = list(set(small_item))
    for index, value in enumerate(sm):
        #이미 분할 되었으면 break
        if value[0].isnumeric():
            break
        tp_1 = sm[index]
        sm[index]= (str(small_item.count(value)))+tp_1
        
    johab = match_item(final_set, sm)
    

    i.append(sm)
    i.append(johab[0])
    i[1].sort(reverse=True)

In [155]:
small_itemlist

['B.F. 대검',
 '거인의 허리띠',
 '곡궁',
 '뒤집개',
 '쇠사슬 조끼',
 '쓸데없이 큰 지팡이',
 '여신의 눈물',
 '연습용 장갑',
 '음전자 망토']

In [156]:
def make_itemlist(arr,items):
    item_ct = []
    for i in items:
        check = 0
        for j in arr:
            ct = int(j[0])
            if i == j[1:]:
                item_ct.append(ct)
                check = 1
                break
        if check == 0:
            item_ct.append(0)
    return item_ct

In [157]:
for i,v in enumerate(cp):
    jh = str(v.pop())
    ar = make_itemlist(v.pop(),small_itemlist)
    rank = int(v.pop())
    v.append(rank)
    v = list(itertools.chain(v,ar))
    v.append(jh)
    cp[i] = v

In [158]:
for i in cp[:10]:
    print(len(i))

11
11
11
11
11
11
11
11
11
11


In [175]:
cp = np.array(cp)

In [176]:
columns = list(itertools.chain(['rank'],small_itemlist,['조합']))

In [177]:
df = pd.DataFrame(cp,columns=columns)

In [178]:
df.head()

,rank,B.F. 대검,거인의 허리띠,곡궁,뒤집개,쇠사슬 조끼,쓸데없이 큰 지팡이,여신의 눈물,연습용 장갑,음전자 망토,조합
0,6,2,0,3,0,2,1,1,4,4,조율자결투가
1,6,2,0,2,0,2,1,2,2,1,명사수귀감
2,3,3,1,3,0,2,3,3,3,0,신성조율자
3,2,1,4,5,0,1,0,2,5,6,그림자닌자
4,3,3,0,1,0,1,3,3,3,2,달빛사냥꾼


In [179]:
df.to_csv('itemlist_ver1.csv',index=False)

## 메인 조합을 불러오기

In [10]:
def find_meta(item_dict):

#     driver = wd.Chrome(executable_path="/mnt/c/Users/shurona/Documents/chromedriver.exe")
    url = 'https://lolchess.gg/meta'
#     driver.get(url)
    hdr = {'Accept-Language' : 'Ko_KR'}
    req = requests.get(url, headers = hdr)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    
    tp = soup.find("div",{"class" : "container-full"})
    
    #최종 저장 리스트
    final_set = []
    
    #각각의 내용을 저장할 장소
    title = {}
    Configuration = []
    champ_list = []
    items_list = []
    main_johap = []
    small_title = []
    #페이지의 메타를 정리해놓은 표에서 시너지 별로 표를 정리한다. 
    for i,v in enumerate(tp.select('div[class=guide-meta__deck]')):
        #sub 리스트 
        sub = []
        
        #공백 지우기 
        del_blank = v.find('div',{"class" : "guide-meta__deck__column name mr-3"}).text.strip()
    
        #데이터를 저장할 리스트들
        johap_name=''
        
        #조합 모으기
        for find_togle in v.select(".tft-hexagon-image"):
            num = (find_togle.get('data-tooltip-url')[-1])
            word = (find_togle.find('img', alt = True)['alt'])
            Configuration.append(str(num) + " " + word)
        
        #챔프 모으기 
        for find_champ in v.select(".tft-champion"):
            name = find_champ.find('img', alt=True)['alt']
            champ_list.append(name)

        #아이템 모으기
        for find_item in v.select(".tft-items"):
            t = find_item.select('img')
            if len(t) == 0:
                continue
            for trash in t:
                trash['alt']
        
        count = 0
        for index,value in enumerate(del_blank):
        
            if ord('가') <= ord(value) <= ord('힣'):
                if count == 0:
                    start = index
                count=1
            else:
                if count == 1:
                    johap_name = johap_name+(''.join(del_blank[start:index]))
                    count = 0
        if count == 1:
#             small_title.append(del_blank[start:])
            johap_name = johap_name+(del_blank[start:])
            
        small_title.append(johap_name)
            
        for item in v.find_all('div',{'class':'tft-items'}):
            items = item.find_all('img')
            if len(items) == 0:
                continue
            for tem in items:
                items_list.append(tem.get('alt'))
                
                
        sub.append(Configuration[:])
        sub.append(items_list[:])
        main_johap.append(sub[:])
#         title['구성'] = Configuration[:]
#         title['챔피언'] = champ_list[:]
#         title['아이템'] = items_list[:]

        final_set.append(title.copy())
        

        champ_list.clear()
        Configuration.clear()
        items_list.clear()
        title.clear()
        sub.clear()

    main_johap.append(small_title)
    return main_johap

In [11]:
final_set = find_meta(item_list)

In [12]:
for i in final_set[:14]:
    before = i.pop()
    after = decomposition_item(item_list,before)
    af = list(set(after))
    for index, value in enumerate(af):
        if value[0].isnumeric():
            break
        af_tp = af[index]
        af[index] = str(after.count(value))+af_tp
    i.append(af)

## 셀레니움을 사용하여서 상위 티어의 전적 가져오기

In [8]:
#decks > div > section > div.col-12.col-xl-9

#soup를 받아오는 함수
def receive_soup(url, hdr):
    req = requests.get(url,headers = hdr)
    html = req.text
    return(BeautifulSoup(html, 'html.parser'))
    
#url을 받아오면 여기서 s
def get_info(url,hdr,driver):
    req = requests.get(url,headers = hdr)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    history = soup.find('div',{'class' : 'profile__match-history-v2__filters'}).select('a')[1]['href']
    url = history
    driver.get(url)
    for i in driver.find_elements_by_class_name('toggle-detail'):
        i.click()
        time.sleep(1)
    time.sleep(2)
    sp = BeautifulSoup(driver.page_source, 'html.parser')
    return sp
    
options = wd.ChromeOptions()
options.add_argument('--window-size=1920,1080')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = wd.Chrome(executable_path="/mnt/c/Users/shurona/Documents/chromedriver.exe", options=options)    
url = 'https://lolchess.gg/leaderboards'
hdr = {'Accept-Language' : 'Ko_KR', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
req = requests.get(url, headers = hdr)
html = req.text
soup = BeautifulSoup(html, 'html.parser')
for i in soup.find('tbody').select('a'):
    sp = get_info(i['href'],hdr,driver)
    break

In [12]:
import itertools
count  = 1
for i in sp.find_all('div',{'class' : 'profile__match-history-v2__item__detail'}):
    for score, value in enumerate(i.find('tbody').find_all('tr')):
        print(count)
        count += 1
        arr=[]
        need_item=[]

        traits = value.select_one('td[class=traits]')
        champions = value.select_one('td[class=champions]')  
        for kinds in traits.select('.tft-hexagon-image'):
            print(kinds.find('img')['alt'], end = ' ')
        print()
        for champ in champions.select('.champions__image'):
            print(champ.find('img')['alt'], end = ' ')
        print()
        for items in champions.select('.champions__items'):
            if items.find('img') != None:
                for i in items.select('img'):
                    arr.append(i['data-original-title'])
#         print(arr)
        for big in arr:
            if big in item_list.keys():
                for divide in item_list[big]:
                    need_item.append(divide)
        print(list(set(need_item)))
        
        
        break
        print()

1
1 황제 5 황혼 2 귀감 2 조율자 2 신비술사 2 명사수 1 추방자 1 닌자 
니달리 쓰레쉬 쉔 진 카시오페아 리븐 아지르 요네 
['쓸데없이 큰 지팡이', '쇠사슬 조끼', '곡궁', '거인의 허리띠', '뒤집개', 'B.F. 대검', '음전자 망토', '여신의 눈물']
2
6 명사수 1 황제 2 황혼 2 신비술사 1 추방자 
니달리 베인 티모 징크스 진 카시오페아 아지르 요네 질리언 
['쓸데없이 큰 지팡이', '쇠사슬 조끼', '연습용 장갑', '곡궁', '거인의 허리띠', 'B.F. 대검', '음전자 망토', '여신의 눈물']
3
1 우두머리 4 암살자 3 달빛 3 나무정령 2 싸움꾼 2 현혹술사 1 닌자 
마오카이 다이애나 리산드라 파이크 아칼리 카타리나 세트 이즈리얼 
['쓸데없이 큰 지팡이', '쇠사슬 조끼', '곡궁', '거인의 허리띠', '뒤집개', 'B.F. 대검', '음전자 망토']
4
6 신비술사 1 황제 3 요술사 2 황혼 2 현혹술사 1 닌자 
잔나 럭스 유미 쉔 카시오페아 릴리아 아지르 이즈리얼 질리언 
['쓸데없이 큰 지팡이', '쇠사슬 조끼', '연습용 장갑', '곡궁', '거인의 허리띠', '뒤집개', 'B.F. 대검', '음전자 망토', '여신의 눈물']
5
1 우두머리 1 황제 6 대장군 3 귀감 2 암살자 2 싸움꾼 
니달리 바이 자르반 4세 파이크 카타리나 리븐 세트 아지르 
['쓸데없이 큰 지팡이', '쇠사슬 조끼', '연습용 장갑', '거인의 허리띠', 'B.F. 대검', '음전자 망토', '여신의 눈물']
6
1 우두머리 3 사냥꾼 3 나무정령 3 요술사 2 조율자 2 싸움꾼 2 신비술사 1 추방자 1 닌자 
룰루 쉔 워윅 애쉬 릴리아 요네 질리언 세트 
['쓸데없이 큰 지팡이', '쇠사슬 조끼', '연습용 장갑', '곡궁', '거인의 허리띠', '뒤집개', 'B.F. 대검', '음전자 망토', '여신의 눈물']
7
8 결투가 2 추방자 1 황제 3 신성 
피오라 야스오 잭스 신 짜오 칼리스타 질리언 리 신 

In [13]:
a = [1,2,3]
lambda a : a+2

<function __main__.<lambda>(a)>

In [181]:
# driver = wd.Chrome(executable_path="/mnt/c/Users/shurona/Documents/chromedriver.exe")
# url = 'https://lolchess.gg/decks'
# driver.get(url)
# driver.find_element_by_css_selector('.decks-header__button').click()
# time.sleep(5)
# html = driver.page_source